In [37]:
import sys
import fitparse
import pandas as pd

In [4]:
fn = '/Users/sentient-asoellinger/code/garmin_fun/examples/2628314301.fit'
ff = fitparse.FitFile(fn)

In [12]:
a = ff.get_messages('record')

In [59]:
data = []

for record in ff.get_messages('record'):

    data.append(record.get_values())

data = pd.DataFrame(data)
data.index = data.timestamp

In [69]:
data['single_offset'] = data.timestamp.shift(1)
data['period'] =  data.timestamp - data.single_offset
data.period.describe()

count                       911
mean     0 days 00:00:05.425905
std      0 days 00:00:02.871372
min             0 days 00:00:01
25%             0 days 00:00:03
50%             0 days 00:00:06
75%             0 days 00:00:08
max             0 days 00:00:18
Name: period, dtype: object